# Размещение баннеров
Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

In [52]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift

In [47]:
df = pd.read_csv('/home/joosja/Загрузки/fsq/umn_foursquare_datasets/checkins.csv', sep='|', skipinitialspace=True, skiprows=[1], index_col=0)
df.head()

/home/joosja/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,user_id,venue_id,latitude,longitude,created_at
id,,,,,
984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


In [48]:
df.columns = [column.strip() for column in df.columns]
df.dropna(inplace=True)
df.drop(['user_id', 'venue_id', 'created_at'], axis=1, inplace=True)
df.head()

,latitude,longitude
id,,
984222,38.895112,-77.036366
984234,33.800745,-84.410520
984291,45.523452,-122.676207
984318,40.764462,-111.904565
984232,33.448377,-112.074037


In [62]:
df = df.iloc[:100000]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 984222 to 127284
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   latitude   100000 non-null  float64
 1   longitude  100000 non-null  float64
 2   cluster    100000 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 3.1+ MB


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

In [76]:
%%time
MS = MeanShift(bandwidth = 0.1, n_jobs=-1)
MS.fit(df)

CPU times: user 15.7 s, sys: 3.6 s, total: 19.3 s
Wall time: 1min 14s


MeanShift(bandwidth=0.1, n_jobs=-1)

In [88]:
labels = MS.labels_
cluster_centers = MS.cluster_centers_

[[ 4.07176807e+01 -7.39919008e+01  0.00000000e+00]
 [ 3.34484105e+01 -1.12074004e+02  1.00000000e+00]
 [ 3.34512309e+01 -1.11917520e+02  2.00000000e+00]
 ...
 [-3.78229826e+01  1.45181190e+02  3.38700000e+03]
 [-4.12924945e+01  1.74773235e+02  3.38800000e+03]
 [-4.50311622e+01  1.68662644e+02  3.38900000e+03]]


In [89]:
labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

In [94]:
counts = dict()
for i in range(len(labels)):
    if labels[i] in counts:
        counts[labels[i]]+=1
    else:
        counts[labels[i]]=1

In [97]:
interesting_labels = [label for label in labels_unique if counts[label]>15]
points = [cluster_centers[i] for i in range(len(cluster_centers)) if interesting_labels.count(labels[i])>0]

In [98]:
offices = []
offices.append([33.751277, -118.188740])
offices.append([25.867736, -80.324116])
offices.append([51.503016, -0.075479])
offices.append([52.378894, 4.885084])
offices.append([39.366487, 117.036146])
offices.append([-33.868457, 151.205134])

In [99]:
best_points=[]
dist=[]
for point in points:
    best_dist=1000000
    for office in offices:
        d = np.sqrt((point[0]-office[0])*2+(point[1]-office[1])*2)
        if d<best_dist:
            best_dist=d
    dist.append(best_dist)

dist = np.array(dist)
idxs = np.argsort(dist)[:20]

<ipython-input-99-1580a9d52866>:6: RuntimeWarning: invalid value encountered in sqrt
  d = np.sqrt((point[0]-office[0])*2+(point[1]-office[1])*2)


In [101]:
points=np.array(points)
for point in points[idxs]:
    print("%f,%f" % (point[0], point[1]))

37.556695,-121.994125
34.022722,-118.459223
52.372964,4.892317
32.793784,-117.227353
-33.860630,151.204776
34.257607,-88.703386
52.921899,-1.475642
25.789202,-80.213114
37.693034,-122.097845
38.232417,-122.636652
47.015240,10.293670
48.849920,2.637041
34.206461,-118.579062
38.334893,-122.704326
33.805774,-118.155494
56.837814,60.596842
33.211757,-87.569979
10.767549,106.681102
27.929377,-82.270639
18.796464,98.660059
